In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
import os
import shutil

def organize_images(base_dir, src_folder):
    src_path = os.path.join(base_dir, src_folder)
    images = [img for img in os.listdir(src_path) if img.endswith(('.png', '.jpg', '.jpeg'))]
    classes = set()

    # Create directories for classes and organize images
    for img in images:
        class_name = img.split('.')[0]
        classes.add(class_name)
        class_dir = os.path.join(base_dir, class_name)
        
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        
        # Move file to new directory
        src_img = os.path.join(src_path, img)
        dest_img = os.path.join(class_dir, img)
        shutil.move(src_img, dest_img)

    print(f"Organized {len(images)} images into {len(classes)} classes.")

# Usage
base_directory = '/Users/sabahatatta/Documents/Projects/Chicken-Disease-Classification/Chicken-Disease-Classification/artifacts/data_ingestion'  # Update this path
source_folder = 'Train'
organize_images(base_directory, source_folder)


In [ ]:
from dataclasses import dataclass
from pathlib import Path

In [ ]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Train")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [ ]:
import time

In [ ]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from tensorflow.python.keras.optimizers import adam_v2
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense


In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def update_model_for_multiclass(self, num_classes=9):
            # Get the output from the second-to-last layer
            x = self.model.layers[-2].output  # This should be a tensor

            # Ensure x can be used as an input to a Dense layer (check it's not a list or tuple)
            if isinstance(x, list):
                x = x[0]  # This is a simplification, might need more complex handling depending on your model structure

            # Add a new Dense layer with the correct number of units and softmax activation
            predictions = Dense(num_classes, activation='softmax')(x)

            # Create a new model based on the modifications
            self.model = Model(inputs=self.model.input, outputs=predictions)

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path 
        )

        self.update_model_for_multiclass(num_classes=9)
        optimizer = tf.keras.optimizers.Adam()
        self.model.compile(
        optimizer= optimizer,  
        loss='categorical_crossentropy',  # Adjust the loss function based on your specific task
        metrics=['accuracy']  # Include any metrics you are interested in
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode="categorical"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        print("Initializing validation generator...")
        print(f"Directory: {self.config.training_data}")
        print(f"Subset: Validation")
        print(f"Class mode: {dataflow_kwargs['class_mode']}")
        print("Initializing training generator...")
        print(f"Directory: {self.config.training_data}")
        print(f"Subset: Training")
        print(f"Class mode: {dataflow_kwargs['class_mode']}")     
                   

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model_save_path = str(path.with_suffix('.keras'))  # Ensure the path ends with '.keras'
        model.save(model_save_path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [ ]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

In [ ]:
# Check if the generators are yielding data
try:
    x_sample, y_sample = next(training.train_generator)
    print("Train generator is set up correctly.")
except StopIteration:
    print("Train generator is empty.")
except Exception as e:
    print(f"Error with train generator: {e}")

try:
    x_val_sample, y_val_sample = next(training.valid_generator)
    print("Validation generator is set up correctly.")
except StopIteration:
    print("Validation generator is empty.")
except Exception as e:
    print(f"Error with validation generator: {e}")


In [ ]:
print("Training with the following configuration:")
print(f"Train generator samples: {training.train_generator.samples}")
print(f"Validation generator samples: {training.valid_generator.samples}")
print(f"Steps per epoch: {training.steps_per_epoch}")
print(f"Validation steps: {training.validation_steps}")
print(f"Epochs: {training.config.params_epochs}")

# If there is a possibility of a conditional assignment failing, check those conditions:
if not hasattr(training, 'model'):
    print("Model is not set. Please check model initialization.")
elif not training.model:
    print("Model is not initialized.")

try:
    training.model.fit(
        training.train_generator,
        epochs=training.config.params_epochs,
        steps_per_epoch=training.steps_per_epoch,
        validation_data=training.valid_generator,
        validation_steps=training.validation_steps,
        callbacks=training.callback_list
    )
except ValueError as e:
    print(f"Caught ValueError during model fitting: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
